In [0]:
dbutils.widgets.text("p_environment", "")
dbutils.widgets.text("p_run_type", "")
dbutils.widgets.text("p_internal_product_id", "")
dbutils.widgets.text("p_source_server_name", "")
dbutils.widgets.text("p_source_database_name", "")
dbutils.widgets.text("p_internal_client_id", "")
dbutils.widgets.text("p_internal_facility_id", "")

In [0]:
%run ../classhandlers/UtilityHandler

In [0]:
required_parameters = ["p_environment", "p_internal_product_id", "p_source_server_name"]
run_parameters_dict = NotebookUtils.get_parameters_dict(dbutils.notebook.entry_point.getCurrentBindings())

NotebookUtils.validate_parameters(run_parameters_dict, required_parameters)

#convert to int
run_parameters_dict["p_internal_product_id"] = int(run_parameters_dict["p_internal_product_id"])
run_parameters_dict["p_internal_client_id"] = int(run_parameters_dict["p_internal_client_id"]) if run_parameters_dict["p_internal_client_id"] else None
run_parameters_dict["p_internal_facility_id"] = int(run_parameters_dict["p_internal_facility_id"]) if run_parameters_dict["p_internal_facility_id"] else None

# convert Run Type
run_parameters_dict["p_run_type"] = run_parameters_dict["p_run_type"].lower() if run_parameters_dict["p_run_type"] else "both"

print(f"Run Parameters:\n{run_parameters_dict}")

In [0]:
%run ../classhandlers/GatewayManagementHandler

In [0]:
%run ../classhandlers/IngestionManagementHandler

In [0]:
import time

In [0]:
gw_obj = CreateSqlGatewayPipeline(run_parameters_dict["p_environment"], 
                            run_parameters_dict["p_internal_product_id"], 
                            run_parameters_dict["p_source_server_name"], 
                            run_parameters_dict["p_source_database_name"], 
                            run_parameters_dict["p_internal_client_id"], 
                            run_parameters_dict["p_internal_facility_id"]
                            )

ingst_obj = CreateSqlIngestionPipeline(run_parameters_dict["p_environment"], 
                            run_parameters_dict["p_internal_product_id"], 
                            run_parameters_dict["p_source_server_name"], 
                            run_parameters_dict["p_source_database_name"], 
                            run_parameters_dict["p_internal_client_id"], 
                            run_parameters_dict["p_internal_facility_id"]
                            )


In [0]:
return_ingst_dict = {}
return_gw_dict = {}
try:
    if run_parameters_dict["p_run_type"] == "ingest":
        return_ingst_dict = ingst_obj.process()
    elif run_parameters_dict["p_run_type"] == "gateway":
        return_gw_dict = gw_obj.process()
    else:
        return_gw_dict = gw_obj.process()
        if 'Success' in  return_gw_dict and return_gw_dict["Success"] > 0:
            #time.sleep(300)
            return_ingst_dict = ingst_obj.process()
except Exception as e:
    print(e)            
finally:
    print ("\n\n","Gateway Implementation:", return_gw_dict,"\n\n")
    print ("Ingestion Implementation:", return_ingst_dict,"\n\n")

    if ('ExecutionFailed' in return_gw_dict and return_gw_dict['ExecutionFailed'] > 0) or \
    ('ExecutionFailed' in return_ingst_dict and return_ingst_dict['ExecutionFailed'] > 0) :
        raise Exception("Execution Failed for one or more pipelines")
    